In [ ]:
!pip install gradio pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!pip install --upgrade gradio

In [ ]:
!pip install --upgrade gradio transformers

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=3bebe21dd41019256b6acdd4df86216ebc5c925afacae1a2712fb2b6d50b3463
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!python -m spacy download xx_ent_wiki_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.




---



In [ ]:
import spacy
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
from langdetect import detect
import re

# Load the multilingual NLP model
nlp = spacy.load("xx_ent_wiki_sm")  # Multilingual spaCy model

def extract_keywords(user_input):
    """Extract keywords from user input using spaCy."""
    doc = nlp(user_input)
    keywords = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return keywords

class Chatbot:
    def __init__(self, dataset, book_data):
        """Initialize the chatbot with dataset and book data."""
        self.dataset = dataset
        self.book_data = book_data
        self.vectorizer, self.X = self.prepare_vectorizer()

    def prepare_vectorizer(self):
        """Create and fit a TF-IDF vectorizer on the provided corpus."""
        corpus = [entry["User"] for entry in self.dataset]
        if not corpus:
            raise ValueError("Corpus for TF-IDF Vectorizer is empty.")
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(corpus)
        return vectorizer, X

    def get_response(self, user_input):
        """Get the chatbot's response based on user input."""
        if not user_input.strip():
            return "Please provide a valid input."
        try:
            user_vec = self.vectorizer.transform([user_input])
            similarities = cosine_similarity(user_vec, self.X)
            best_match_index = similarities.argmax()
            if similarities[0, best_match_index] > 0.5:
                return self.dataset[best_match_index]["Chatbot"]
        except Exception as e:
            print(f"Error during response generation: {e}")
            return "Oops, something went wrong while processing your request. Can you try again?"
        return "Hmm, I couldn’t find anything related to that. Want to try rephrasing?"

    def recommend_books(self, user_input, filters={}):
        """Recommend books based on user input and filters."""
        keywords = extract_keywords(user_input)
        print(f"Extracted Keywords: {keywords}")  # Debug output
        try:
            # Match titles with stricter word boundaries using regex
            keyword_pattern = '|'.join(f"\\b{re.escape(keyword)}\\b" for keyword in keywords)
            matched_books = self.book_data[
                self.book_data['Title'].str.contains(keyword_pattern, case=False, na=False, flags=re.UNICODE)
            ]

            print(f"Matched Books Before Filters: {matched_books}")  # Debug output

            for key, value in filters.items():
                if value:
                    matched_books = matched_books[matched_books[key].str.contains(value, case=False, na=False)]

            print(f"Matched Books After Filters: {matched_books}")  # Debug output

            if not matched_books.empty:
                # Only show books where titles contain all keywords
                filtered_books = matched_books[matched_books['Title'].apply(lambda title: all(re.search(f"\\b{re.escape(keyword)}\\b", title, re.IGNORECASE) for keyword in keywords))]

                if not filtered_books.empty:
                    response = "<div style='max-height: 300px; overflow-y: auto;'>"
                    response += "<p>Here are some books you might like:</p><ul>"
                    for _, row in filtered_books.iterrows():
                        response += f"{row['Title']} ({row['Status']}, located at {row['Location']})\n"
                    return response
                else:
                    return "<p>Sorry, I couldn’t find any books matching all your keywords. Maybe try simplifying your query?</p>"
        except Exception as e:
            print(f"Error during book recommendation: {e}")
            return "<p>Yikes, something went wrong while finding books for you. Mind trying again?</p>"

        return "<p>Sorry, I couldn’t find any books matching your request. Maybe try different keywords?</p>"

# Gradio interface function
def chatbot_ui(user_input):
    """Interface function to get a response from the chatbot or recommend books."""
    try:
        filters = {}
        if "recommend" in user_input.lower():
            parts = user_input.lower().split(',')
            keywords = parts[0].replace("recommend", "").strip()
            for part in parts[1:]:
                key, value = part.split(":")
                filters[key.strip()] = value.strip()
            response = chatbot.recommend_books(keywords, filters)
        else:
            response = chatbot.get_response(user_input)
        return response
    except Exception as e:
        print(f"Error: {e}")  # Log the error in the terminal
        return f"<p>Oh no! Something went wrong: {e}. Can you try again?</p>"

# Load datasets and validate
file_path = "formatted_chatbot_data.json"
try:
    with open(file_path, 'r') as f:
        dataset = json.load(f)
        if not dataset:
            raise ValueError("Chatbot dataset is empty or not loaded correctly.")
except Exception as e:
    print(f"Error loading chatbot dataset: {e}")
    raise

book_data_path = "lspu_library_dataset - Sheet1.csv"
try:
    book_data = pd.read_csv(book_data_path)
    if book_data.empty:
        raise ValueError("Book data is empty or not loaded correctly.")
except Exception as e:
    print(f"Error loading book data: {e}")
    raise

# Initialize chatbot and Gradio interface
chatbot = Chatbot(dataset, book_data)

chatbot_interface = gr.Interface(
    fn=chatbot_ui,
    inputs=gr.Textbox(lines=3, placeholder="Type your question or request book recommendations...", label="Your Message"),
    outputs=gr.Textbox(label="Response"),
    title="Libby the Library Assistant",
    description="Ask about books or get recommendations."
)

chatbot_interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c0e56fe2f45ad459be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
